In [336]:
"""CELL 1
builds and returns a database session
local assumes a psql instance in a local docker container
only postgres database is supported for configuration_application at this time
"""
"""
gets env-based configuration secret
returns a session to the configuration db
for dev env it pre-populates the database with helper and seed data
"""
from core.helpers.session_helper import SessionHelper
session = SessionHelper().session

2019-07-24 19:51:25,576 - core.helpers.session_helper.SessionHelper - INFO - Creating session for dev environment...
2019-07-24 19:51:25,599 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Generating administrator mocks.
2019-07-24 19:51:25,603 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Done generating administrator mocks.
2019-07-24 19:51:25,604 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Generating pharmaceutical company mocks.
2019-07-24 19:51:25,607 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Done generating pharmaceutical company mocks.
2019-07-24 19:51:25,608 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Generating brand mocks.
2019-07-24 19:51:25,613 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Done generating brand mocks.
2019-07-24 19:51:25,615 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Generating segment mocks.
2019-07-24 19:51:2

In [337]:
"""
************ CONFIGURATION - PLEASE TOUCH **************
Pipeline Builder configuration: creates configurations from variables specified here!!
This cell will be off in production as configurations will come from the configuration postgres DB.
"""
"""
PIPELINE STATE:

raw-->ingest-->master-->enhance-->enrich-->metrics-->dimensional

"""
# config vars: this dataset
config_pharma = "sun" # the pharmaceutical company which owns {brand}
config_brand = "ilumya" # the brand this pipeline operates on
config_state = "master" # the state this transform runs in
config_name = "master_patient_status" # the name of this transform, which is the name of this notebook without .ipynb

# input vars: dataset to fetch. Recall that a contract published to S3 has a key format branch/pharma/brand/state/name
input_pharma = "sun"
input_brand = "ilumya"
input_state = "ingest"
input_name = "symphony_health_association_ingest_column_mapping"
input_branch = "sun-extract-validation"
# None
# "sun-extract-validation" # if None, input_branch is automagically set to your working branch



In [338]:
"""
************ SETUP - DON'T TOUCH **************
Populating config mocker based on config parameters...
"""
import core.helpers.pipeline_builder as builder

ids = builder.build(config_pharma, config_brand, config_state, config_name, session)

transform_id = ids[0]
run_id = ids[1]

2019-07-24 19:51:32,092 - core.logging - DEBUG - Adding/getting mocks for specified configurations...
2019-07-24 19:51:32,122 - core.logging - DEBUG - Done. Creating mock run event and committing results to configuration mocker.


In [339]:
"""
************ SETUP - DON'T TOUCH **************
This section imports data from the configuration database
and should not need to be altered or otherwise messed with. 
~~These are not the droids you are looking for~~
"""
from core.constants import BRANCH_NAME, ENV_BUCKET
from core.helpers.session_helper import SessionHelper
from core.models.configuration import Transformation
from dataclasses import dataclass
from core.dataset_contract import DatasetContract

db_transform = session.query(Transformation).filter(Transformation.id == transform_id).one()

@dataclass
class DbTransform:
    id: int = db_transform.id ## the instance id of the transform in the config app
    name: str = db_transform.transformation_template.name ## the transform name in the config app
    state: str = db_transform.pipeline_state.pipeline_state_type.name ## the pipeline state, one of raw, ingest, master, enhance, enrich, metrics, dimensional
    branch:str = BRANCH_NAME ## the git branch for this execution 
    brand: str = db_transform.pipeline_state.pipeline.brand.name ## the pharma brand name
    pharmaceutical_company: str = db_transform.pipeline_state.pipeline.brand.pharmaceutical_company.name # the pharma company name
    publish_contract: DatasetContract = DatasetContract(branch=BRANCH_NAME,
                            state=db_transform.pipeline_state.pipeline_state_type.name,
                            parent=db_transform.pipeline_state.pipeline.brand.pharmaceutical_company.name,
                            child=db_transform.pipeline_state.pipeline.brand.name,
                            dataset=db_transform.transformation_template.name)


# CORE Cartridge Notebook::[master_patient_status]
![CORE Logo](assets/coreLogo.png) 

---
## Keep in Mind
Good Transforms Are...
- **singular in purpose:** good transforms do one and only one thing, and handle all known cases for that thing. 
- **repeatable:** transforms should be written in a way that they can be run against the same dataset an infinate number of times and get the same result every time. 
- **easy to read:** 99 times out of 100, readable, clear code that runs a little slower is more valuable than a mess that runs quickly. 
- **No 'magic numbers':** if a variable or function is not instantly obvious as to what it is or does, without context, maybe consider renaming it.

## Workflow - how to use this notebook to make science
#### Data Science
1. **Document your transform.** Fill out the _description_ cell below describing what it is this transform does; this will appear in the configuration application where Ops will create, configure and update pipelines. 
1. **Define your config object.** Fill out the _configuration_ cell below the commented-out guide to define the variables you want ops to set in the configuration application (these will populate here for every pipeline). 
2. **Build your transformation logic.** Use the transformation cell to do that magic that you do. 
![caution](assets/cautionTape.png)

# Configuration

In [369]:
""" 
CONFIGURATION 
********* VARIABLES - PLEASE TOUCH ********* 
This section defines what you expect to get from the configuration application 
in a single "transform" object. Define the vars you need here, and comment inline to the right of them 
for all-in-one documentation. 
Engineering will build a production "transform" object for every pipeline that matches what you define here.

@@@ FORMAT OF THE DATA CLASS IS: @@@ 

<variable_name>: <data_type> #<comment explaining what the value is to future us>
e.g.
class Transform(DbTransform):
    some_ratio: float
    site_name: str

~~These ARE the droids you are looking for~~
"""
"""
imports
"""
import pandas as pd
# getting a logger through the core project helper function
from core.logging import get_logger
logger = get_logger(f"core.transforms.{transform.state}.{transform.name}")

 
class Transform(DbTransform):
    '''
    YOUR properties go here!!
    Variable properties should be assigned to the exact name of
    the transformation as it appears in the Jupyter notebook filename.
    ''' 

    ## cols of interest
    ## Field	     Description	Data Type / Format	Comments	Required	            OLD Name in Data Model	New Name in Data Model	<-- Map to	BIPI	Sun	Alkermes - Status
    ## Status Code	Status Code	    X(20)		                    Required if Available	status	                status				
    col_status: str           
    
    
    def master_patient_status(self,df):
        try:        
            go = True # assume things will work
           
            dffails = pd.DataFrame() # initialize df for fails
            
            # master data golden status
            #working draft] Gold Domain of patient status
            logger.info('try:')
            status_dict = {}
            status_dict[1]='ACTIVE'
            status_dict[2]='CANCELLED'
            status_dict[3]='DISCONTINUED'
            status_dict[4]='DENIED'
            status_dict[5]='PENDING'
            
            # store the golden values in a list
            status_list = list(status_dict.values())           
            
            # log metadata 
            
            logger.info('Gold Domain List:{}'.format(status_list))  
            
            # df in
            logger.info('df in {}'.format(df.head()))
            dfSize = df.size
            dfShape = df.shape
            logger.info('input df  size: {} shape: {} {}'.format(dfSize,dfShape[0],dfShape[1])) 
            
            # am I expecting certain column names? YES 
            statusColNameExpected = 'status_code'
            
            logger.info('expecting column name for patient status as:{}'.format(statusColNameExpected))
            columnNamesArr = df.columns.values.tolist()
            logger.info('df column names:{}'.format(columnNamesArr))
            
            if statusColNameExpected in columnNamesArr:
                # can we assume anything about raw to ingest data ? 
                # apply Upper Case to col values of interest
                df[statusColNameExpected]= df[statusColNameExpected].apply(lambda x: x.upper() if x is not None else x)   
                # apply strip  to col values of interest
                df[statusColNameExpected]= df[statusColNameExpected].apply(lambda x: x.strip() if x is not None else x)
                # what fails
                dffail = df[~df[statusColNameExpected].isin(status_list)]
                # apply master selection for the column of interest
                # what passes
                df = df[df[statusColNameExpected].isin(status_list)]
                
                # meta data log for what comes out of the function pass df
                dfOutSize = df.size
                dfOutShape = df.shape
                logger.info('out  df  size: {} shape: {} {}'.format(dfOutSize,dfOutShape[0],dfOutShape[1])) 
                
                # meta data log for what comes out of the function fail df
                logger.info('df fail {}'.format(dffail.head()))
                
                dffailSize = dffail.size
                dffailShape = dffail.shape
                logger.info('fail df  size: {} shape: {} {}'.format(dffailSize,dffailShape[0],dfFailShape[1]))                 
            else:
                go = False # something did not work
                logger.exception('expecting column name for patient status if/else exception raise')
                raise Exception("master_patient_status try if else saletermColNameExpected in columnNamesArr")              
        except Exception as e:
            go = False # something did not work
            logger.exception("exception:".format(e))
            pass
            #raise Exception("master_patient_status"+str(e))
        else:
            pass
        finally:
            pass
        return df,dffails,go
    
             

transform = Transform()



In [351]:
## Please place your value assignments for development here!!
## This cell will be turned off in production and Engineering will set to pull from the configuration application instead
## For the last example, this could look like...
## transform.some_ratio = 0.6
## transform.site_name = "WALGREENS"

transform.col_status = 'status_code'


### Description
What does this transformation do? be specific.

![what does your transform do](assets/what.gif)

## Planned
1. Collect all unique raw patient status instances
2. Auto-map as many raw patient status instances to a defined cleansed data model.
*working draft Gold Domain of patient status:*  

   1. ACTIVE
   
   2. CANCELLED
   
   3. DENIED
   
   4. DISCONTINUED
   
   5. PENDING

3. Process for identifying and manually mapping where auto-map fails.
4. Do not publish un-mapped instances. Drop them, give us the ability to triage and map to gold in a later event.


# Transformation

### *FETCH DATA - TOUCH, BUT CAREFULLY*
#### *This cell will be turned off in production, as the input_contract will be handled by the pipeline.*

In [342]:
"""
************ FETCH DATA - TOUCH, BUT CAREFULLY **************
This cell will be turned off in production, as the input_contract will be handled by the pipeline.
"""
logger.info("FETCH DATA CELL - TOUCH - This cell will be turned off in production, as the input_contract will be handled by the pipeline. ")

# for testing / development only
run_id = 3

if not input_branch:
    input_branch = BRANCH_NAME
input_contract = DatasetContract(branch=input_branch,
                                 state=input_state, 
                                 parent=input_pharma, 
                                 child=input_brand, 
                                 dataset=input_name)
run_filter = []
run_filter.append(dict(partition="__metadata_run_id", comparison="==", values=[run_id]))
# IF YOU HAVE PUBLISHED DATA MULTIPLE TIMES, uncomment the above line and change the int to the run_id to fetch.
# Otherwise, you will have duplicate values in your fetched dataset!

# use unit testing devlopment for now

#df = input_contract.fetch(filters=run_filter)



2019-07-24 19:51:52,132 - core.transforms.master.master_patient_status - INFO - FETCH DATA CELL - TOUCH - This cell will be turned off in production, as the input_contract will be handled by the pipeline. 


## *<font color=grey>Unit test development only*</font>

In [343]:
# temporary cell only for unit test development
import pyarrow.parquet as pq
import s3fs

def pandas_from_parquet_s3(file_path):  
    s3 = s3fs.S3FileSystem()
    df = (
        pq
        .ParquetDataset(file_path, filesystem=s3)
        .read_pandas()
        .to_pandas()
    )    
    return df

In [366]:
# unit test development TEST 1
#df = pandas_from_parquet_s3('ichain-dev/sun-extract-validation/sun/ilumya/ingest/symphony_health_association_ingest_column_mapping/__metadata_run_id=3/90ca3aa7b0bb4246a281591b013ff54e.parquet')
# TEST 2
df = pandas_from_parquet_s3('ichain-dev/sun-extract-validation/sun/ilumya/ingest/symphony_health_association_ingest_column_mapping/__metadata_run_id=3/d7ad974cef284e19aa7b5ac410220b96.parquet')

In [367]:
# unit test development
# before shot unit testing only
dfSize = df.size
dfShape = df.shape
print('size: {} shape: {} {}'.format(dfSize,dfShape[0],dfShape[1])) 

size: 100252 shape: 1412 71


In [233]:
# needed to see the col(s) of interest
# pd.set_option('display.max_columns', 50)

In [353]:
# unit test devlopment
df.head()

,rec_date,pharm_code,pharm_npi,transtype,pharm_transaction_id,trans_seq,ref_source,ref_date,program_id,pharmacy_id,pat_last_name,pat_first_name,pat_dob,pat_gender,pat_addr1,pat_addr2,pat_city,pat_state,pat_zip,dx1_code,dx2_code,status_date,status_code,sub_status,pres_last_name,...,ship_carrier,shiptracking_num,ship_location,ship_address,ship_city,ship_state,ship_zip,has_medical,primary_coverage_type,primary_payer_name,primary_payer_type,secondary_coverage_type,secondary_payer_name,secondary_payer_type,plan_paid_amt,pat_copay,copay_assist_amount,oth_payer_amt,xfer_pharmname,msa_patient_id,msa_patient_bmap,__metadata_run_timestamp,__metadata_app_version,__metadata_output_contract,__metadata_transform_timestamp
0,20181113 09:23:21,CGN,1225030448,COM,10198477,0,DIRECT,20180515 00:00:00,None,238928,None,None,None,None,None,None,None,None,None,C44.91,None,20181113 09:23:21,PENDING,BENEFITS,PATTON,...,FEDEX,None,PATIENT HOME,None,None,None,None,N,PHARMACY,CIGNA,MEDICARE,None,None,None,None,0.00,None,None,None,None,NNNNN,2019-07-01 13:25:07,0.0.11,s3://ichain-dev/sun-extract-validation/sun/ilu...,2019-07-01 13:35:26
1,20181114 09:14:26,CGN,1225030448,COM,10222483,0,DIRECT,20180515 00:00:00,None,238928,None,None,None,None,None,None,None,None,None,C44.91,None,20181114 09:14:26,PENDING,PATIENT CONTACT,PATTON,...,FEDEX,None,PATIENT HOME,None,None,None,None,N,PHARMACY,CIGNA,MEDICARE,None,None,None,None,0.00,None,None,None,None,NNNNN,2019-07-01 13:25:07,0.0.11,s3://ichain-dev/sun-extract-validation/sun/ilu...,2019-07-01 13:35:26
2,20181116 08:53:10,CGN,1225030448,COM,10286875,0,DIRECT,20180515 00:00:00,None,238928,None,None,None,None,None,None,None,None,None,C44.91,None,20181116 08:53:10,PENDING,INVENTORY HOLD,PATTON,...,FEDEX,None,PATIENT HOME,None,None,None,None,N,PHARMACY,CIGNA,MEDICARE,None,None,None,None,0.00,None,None,None,None,NNNNN,2019-07-01 13:25:07,0.0.11,s3://ichain-dev/sun-extract-validation/sun/ilu...,2019-07-01 13:35:26
3,20181119 08:15:54,CGN,1225030448,COM,10314383,0,DIRECT,20180515 00:00:00,None,238928,None,None,None,None,None,None,None,None,None,C44.91,None,20181119 08:15:54,PENDING,PATIENT CONTACT,PATTON,...,FEDEX,None,PATIENT HOME,None,None,None,None,N,PHARMACY,CIGNA,MEDICARE,None,None,None,None,0.00,None,None,None,None,NNNNN,2019-07-01 13:25:07,0.0.11,s3://ichain-dev/sun-extract-validation/sun/ilu...,2019-07-01 13:35:26
4,20181120 09:43:04,CGN,1225030448,COM,10345064,0,DIRECT,20180515 00:00:00,None,238928,None,None,None,None,None,None,None,None,None,C44.91,None,20181120 09:43:04,PENDING,PATIENT CONTACT,PATTON,...,FEDEX,None,PATIENT HOME,None,None,None,None,N,PHARMACY,CIGNA,MEDICARE,None,None,None,None,0.00,None,None,None,None,NNNNN,2019-07-01 13:25:07,0.0.11,s3://ichain-dev/sun-extract-validation/sun/ilu...,2019-07-01 13:35:26


# <font color=orange>**CALL**</font> THE TRANSFORM

In [370]:
### Use the variables above to execute your transformation.
### the final output needs to be a variable named final_dataframe
logger.info("CALL THE TRANSFORM - execute your transformation - the final output needs to be a variable named final_dataframe")

final_dataframe, final_fail, go = transform.master_patient_status(df)

if go==True:
    logger.info("CALL THE TRANSFORM -  go no go = GO")
elif go==False:
    logger.info("CALL THE TRANSFORM -  go no go = NO go")
else:
    go=False
    logger.info("CALL THE TRANSFORM -  go no go = unknown make it NO go")
    
    

2019-07-24 20:21:42,733 - core.transforms.master.master_patient_status - INFO - CALL THE TRANSFORM - execute your transformation - the final output needs to be a variable named final_dataframe
2019-07-24 20:21:42,736 - core.transforms.master.master_patient_status - INFO - try:
2019-07-24 20:21:42,738 - core.transforms.master.master_patient_status - INFO - Gold Domain List:['ACTIVE', 'CANCELLED', 'DISCONTINUED', 'DENIED', 'PENDING']
2019-07-24 20:21:42,770 - core.transforms.master.master_patient_status - INFO - df in             rec_date pharm_code   pharm_npi transtype pharm_transaction_id  \
0  20181113 09:23:21        CGN  1225030448       COM             10198477   
1  20181114 09:14:26        CGN  1225030448       COM             10222483   
2  20181116 08:53:10        CGN  1225030448       COM             10286875   
3  20181119 08:15:54        CGN  1225030448       COM             10314383   
4  20181120 09:43:04        CGN  1225030448       COM             10345064   

  trans_s

### *<font color=grey>unittest python*</font>

In [349]:
import unittest

def ut_shape(final_dataframe,df):
    """
    assertion will change based on coding state
    """
    return final_dataframe.shape == df.shape

class TestNotebook(unittest.TestCase):
    
    def test_ut_shape(self):
        
        self.assertEqual(ut_shape(final_dataframe,df),True)
                
"""
expect ok     9:45 am 2019 23 july since transform function
is doing nothing yet
still expecting ok
expect FAIL   9:23 am 2019 24 july since transform now coded with UNIT TEST 1 as input
expect ok for UNIT TEST 2 file, no fails
"""
# for development only
unittest.main(argv=[''], verbosity= 2, exit=False)        
    

test_ut_shape (__main__.TestNotebook) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


## Publish

In [329]:
## that's it - just provide the final dataframe to the var final_dataframe and we take it from there
if go==True:
    logger.info("PUBLISH - that's it - just provide the final dataframe to the var final_dataframe and we take it from there")
    #transform.publish_contract.publish(final_dataframe, run_id, session)
elif go==False:
    logger.info("PUBLISH -  go no go = NO go -  so DONT publish")
else:
    go=False
    logger.info("PUBLISH -  go no go = unknown make it NO go - so DONT publish")    
session.close()

2019-07-24 19:36:00,552 - core.transforms.master.master_patient_status - INFO - PUBLISH - that's it - just provide the final dataframe to the var final_dataframe and we take it from there
